# Setup

In [ ]:
import contracts
contracts.disable_all()

In [ ]:
import duckietown_world as dw
from duckietown_world.svg_drawing.ipython_utils import ipython_draw_html

In [ ]:
dw.logger.setLevel(50)

Better visualization of output

In [ ]:
%%html
<style>
pre {line-height: 90%}
</style>

# Map representation

Let's load a map and see how data is represented inside:

In [ ]:
m = dw.load_map('4way')
ipython_draw_html(m);

The map is a `DuckietownMap` which is an instance of `PlacedObject`. All spatially situated objects
are instances of this class.

In [ ]:
type(m).mro() # see all superclasses of the object

## Children

The data is arranged in a hierarchy. We can visualize using the function `get_object_tree`. The hierarchy is more deep, but we clip it to 2 levels:

In [ ]:
print(dw.get_object_tree(m, levels=4))

The children are available in the `children` variable:

In [ ]:
m.children

In [ ]:
m.children['tilemap'].children

You can use the notation below to get a child in a compact way:

In [ ]:
lane_segment = m['tilemap/tile-0-1/straight/lane1']

In this case we can see how a Tile has a child `curve_left` with a child `curve` with two children `lane1` and `lane2`:

In [ ]:
tile = m.children['tilemap'].children['tile-0-0']
print(dw.get_object_tree(tile, attributes=False, levels=10))

In [ ]:
print(dw.get_object_tree(tile, attributes=True, levels=10))

In [ ]:
lane1 = tile['curve_left/curve/lane1']
lane2 = tile['curve_left/curve/lane2']
ipython_draw_html(lane1);
ipython_draw_html(lane2);

In [ ]:
curve = tile['curve_left/curve']
print(dw.get_object_tree(curve, attributes=True, spatial_relations=True, levels=10))

In [ ]:
ipython_draw_html(curve);

In [ ]:
lane = tile['curve_left/curve/lane2']._copy()

ipython_draw_html(lane);

In [ ]:
lane.width

In [ ]:
lane.control_points

## Parametrization of lanes

The lane is parametrized with a parameter `beta` that interpolates among the control points.

Here we create an animation of the center point for different `beta`s.

In [ ]:
import numpy as np
npoints = len(lane.control_points)
betas = list(np.linspace(-1, npoints + 1, 20))
print(betas)

In [ ]:
transforms = []
for beta in betas:
    # call the function `center_point` to get the center point (in SE(2))
    p = lane.center_point(beta)
    transform = dw.SE2Transform.from_SE2(p)
    transforms.append(transform)

ground_truth = dw.SampledSequence(betas, transforms)
lane.set_object('traveling-point', dw.PlacedObject(), ground_truth=ground_truth)
ipython_draw_html(lane);

Scrub the timeline to see the animation.

In [ ]:
import geometry as geo
q = geo.SE2_from_translation_angle([+0.05, -0.05], np.deg2rad(20))

In [ ]:
lane.set_object('db18-4', dw.DB18(), ground_truth=dw.SE2Transform.from_SE2(q))
lane_pose = lane.lane_pose_from_SE2(q)

lane.set_object('marker3', dw.PlacedObject(), ground_truth=lane_pose.center_point)

In [ ]:
lane.children

In [ ]:
lane.spatial_relations

In [ ]:
ipython_draw_html(lane);

In [ ]:
assert lane_pose.correct_direction

In [ ]:
assert lane_pose.inside

In [ ]:
assert lane